In [1]:
import numpy
from numpy import *

In [2]:
def loadDataSet(fileName):
    dataMat=[];labelMat=[]
    fr=open(fileName)
    for line in fr.readlines():
        lineArr=line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

In [3]:
def selectJrand(i,m):# i is the index of our first alpha,m is the total number of alphas
    j=i
    while(j==i):#A value is randomly chosen and returned as long as it's not equal to the input i
        j=int(random.uniform(0,m))
    return j

In [4]:
def clipAlpha(aj,H,L):#clips alpha values that are greater than H or less than L
    if aj>H:
        aj=H
    if L>aj:
        aj=L
    return aj

In [5]:
def smoSimple(dataMatIn,classLabels,C,toler,maxIter):#the dataset, the class labels, a constant C, the tolerance,the maximum number of iterations
    dataMatrix=mat(dataMatIn);labelMat=mat(classLabels).transpose()#This makes the row of the class labels correspond to the row of the data matrix
    b=0;m,n=shape(dataMatrix)
    alphas=mat(zeros((m,1)))
    iter=0
    while(iter<maxIter):
        alphaPairsChanged=0#record if the attempt to optimize any alphas worked
        for i in range(m):
            fXi=float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T))+b#this is our prediction of the class
            Ei=fXi-float(labelMat[i])
            #Alphas will be clipped at 0 or C, so if they’re equal to these, they’re “bound” and can’t be increased or decreased, so it’s not worth trying to optimize these alphas.
            if ((labelMat[i]*Ei<-toler) and (alphas[i]<C)) or ((labelMat[i]*Ei>toler) and (alphas[i]>0)):##若alphas[i]的误差很大,则对该数据实例对应的alpha值进行优化  
                j=selectJrand(i,m)#选取与i不同的alpha值  
                #计算样本j的预测类别 
                fXj=float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T))+b
                Ej=fXj-float(labelMat[j])
                #由于python是通过引用的方式传递列表,故复制alphai\j以备后面需要alpha旧值
                alphaIold=alphas[i].copy()
                alphaJold=alphas[j].copy()
                #计算L和H,用于将alpha值调整到0-C之间 
                if labelMat[i]!=labelMat[j]:
                    L=max(0,alphas[j]-alphas[i])
                    H=min(C,C+alphas[j]-alphas[i])
                else:
                    L=max(0,alphas[j]+alphas[i]-C)
                    H=min(C,alphas[j]+alphas[i])
                if L==H:
                    print "L==H";continue
                #Eta is the optimal amount to change alpha[j]
                eta=2.0*dataMatrix[i,:]*dataMatrix[j,:].T-dataMatrix[i,:]*dataMatrix[i,:].T-dataMatrix[j,:]*dataMatrix[j,:].T
                if eta>=0:
                    print "eta>=0";continue
                alphas[j]-=labelMat[j]*(Ei-Ej)/eta
                alphas[j]=clipAlpha(alphas[j],H,L)
                if(abs(alphas[j]-alphaJold)<0.00001):
                    print "j not moving enough";continue
                alphas[i]+=labelMat[j]*labelMat[i]*(alphaJold-alphas[j])
                #set the constant term b for these two alphas
                b1=b-Ei-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T-labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2=b-Ej-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T-labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if 0<alphas[i] and C>alphas[i]:
                    b=b1
                elif 0<alphas[j] and C>alphas[j]:
                    b=b2
                else:
                    b=(b1+b2)/2.0
                alphaPairsChanged+=1
                print "iter: %d i: %d, pairs changed %d" %(iter,i,alphaPairsChanged)
            #check to see if any alphas have been updated; if so you set iter to 0 and continue
            if alphaPairsChanged==0:
                iter+=1
            else:
                iter=0
            print "iteration number:%d"%iter
        return b,alphas

In [14]:
def kernelTrans(X,A,kTup):#The tuple kTup holds information about the kernel. The first argument in the tuple is a string describing what type of kernel should be used
    m,n=shape(X)
    K=mat(zeros((m,1)))
    if kTup[0]=='lin':#linear
        K=X*A.T
    elif kTup[0]=='rbf':#rbf
        for j in range(m):
            deltaRow=X[j,:]-A
            K[j]=deltaRow*deltaRow.T
        K=exp(K/(-1*kTup[1]**2))#element-wise division
    else:
        raise NameError('Houston We Have a Problem -- That Kernel is not recognized')
    return K
        

In [15]:
class optStruct:
    def __init__(self,dataMatIn,classLabels,C,toler,kTup):
        self.X=dataMatIn
        self.labelMat=classLabels
        self.C=C
        self.tol=toler
        self.m=shape(dataMatIn)[0]
        self.alphas=mat(zeros((self.m,1)))
        self.b=0
        #The first column is a flag bit stating whether the eCache is valid, and the second column is the actual E value
        self.eCache=mat(zeros((self.m,2)))#Error cache
        self.K=mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i]=kernelTrans(self.X,self.X[i,:],kTup)

In [19]:
def calcEk(oS,k):#calculates an E value for a given alpha
    fXk=float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k]+oS.b)
    Ek=fXk-float(oS.labelMat[k])
    return Ek

In [20]:
def selectJ(i,oS,Ei):# selects the second alpha,so that we’ll take the maximum step during each optimization
    maxK=-1;maxDeltaE=0;Ej=0
    oS.eCache[i]=[1,Ei]#i为第一个alpha
    validEcacheList=nonzero(oS.eCache[:,0].A)[0]#returns the alphas corresponding to non-zero E values
    if len(validEcacheList)>1:#choose the value that gives you a max- imum change 
        for k in validEcacheList:
            if k==i:
                continue
            Ek=calcEk(oS,k)
            deltaE=abs(Ei-Ek)
            if deltaE>maxDeltaE:
                maxK=k;maxDeltaE=deltaE;Ej=Ek
        return maxK,Ej
    else:#If this is your first time through the loop, you randomly select an alpha
        j=selectJrand(i,oS.m)
        Ej=calcEk(oS,j)
    return j,Ej

In [21]:
def updateEk(oS,k):#calculates the error and puts it in the cache
    Ek=calcEk(oS,k)
    oS.eCache[k]=[1,Ek]

In [22]:
def innerL(i,oS):
    Ei=calcEk(oS,i)
    if ((oS.labelMat[i]*Ei<-oS.tol) and (oS.alphas[i]<oS.C)) or ((oS.labelMat[i]>oS.tol) and (oS.alphas[i]>0)):
        j,Ej=selectJ(i,oS,Ei)
        alphaIold=oS.alphas[i].copy();alphaJold=oS.alphas[j].copy();
        if oS.labelMat[i]!=oS.labelMat[j]:
            L=max(0,oS.alphas[j]-oS.alphas[i])
            H=min(oS.C,oS.C+oS.alphas[j]-oS.alphas[i])
        else:
            L=max(0,oS.alphas[j]+oS.alphas[i]-oS.C)
            H=min(oS.C,oS.alphas[j]+oS.alphas[i])
        if L==H:
            print "L==H"; return 0
        eta=2.0*oS.K[i,j]-oS.K[i,j]-oS.K[j,j]
        if eta>=0:
            print "eta>=0";return 0
        oS.alphas[j]-=oS.labelMat[j]*(Ei-Ej)/eta
        oS.alphas[j]=clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)#Updates Ecache
        if abs(oS.alphas[j]-alphaJold)<0.00001:
            print "j not moving enough";return 0
        oS.alphas[i]+=oS.labelMat[j]*oS.labelMat[i]*(alphaJold-oS.alphas[j])
        updateEk(oS,i)#Updates Ecache
        b1=oS.b-Ei-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i]-oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2=oS.b-Ej-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]-oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if 0<oS.alphas[i] and oS.C>oS.alphas[i]:
            oS.b=b1
        elif 0<oS.alphas[j] and oS.C>oS.alphas[j]:
            oS.b=b2
        else:
            oS.b=(b1+b2)/2.0
        return 1
    else:
        return 0

In [25]:
def smoP(dataMatIn,classLabels,C,toler,maxIter,kTup=('lin',0)):#This is the outer loop where you select the first alpha.
    oS=optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler,kTup)
    iter=0;entireSet=True;alphaPairsChanged=0
    #You’ll exit from the loop whenever the number of iterations exceeds your specified maximum or you pass through the entire set without changing any alpha pairs.
    while (iter<maxIter) and ((alphaPairsChanged>0) or (entireSet)):
        alphaPairsChanged=0
        if entireSet:
            for i in range(oS.m):#goes over any alphas in the dataset
                alphaPairsChanged+=innerL(i,oS)#choose a second alpha and do optimization if possible. A 1 will be returned if any pairs get changed
                print "fullSet, iter: %d i:%d, pairs changed %d"%(iter,i,alphaPairsChanged)
            iter+=1
        else:
            nonBoundIs=nonzero((oS.alphas.A>0)*(oS.alphas.A<C))[0]
            for i in nonBoundIs:#goes over all the non-bound alphas, the values that aren’t bound at 0 or C
                alphaPairsChanged+=innerL(i,oS)
                print "non-bound,iter: %d i:%d,pairs changed %d"%(iter,i,alphaPairsChanged)
            iter+=1
        if entireSet:
            entireSet=False
        elif alphaPairsChanged==0:
            entireSet=True
        print "iteration number:%d"%iter
    return oS.b,oS.alphas

In [26]:
def calcWs(alphas,dataArr,classLabels):
    X=mat(dataArr);labelMat=mat(classLabels).transpose()
    m,n=shape(X)
    w=zeros((n,1))
    for i in range(m):
        w+=multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

In [30]:
def testRbf(k1=1.3):
    dataArr,labelArr=loadDataSet('testSetRBF.txt')
    b,alphas=smoP(dataArr,labelArr,200,0.0001,10000,('rbf',k1))
    dataMat=mat(dataArr);labelMat=mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]#选择alpha大于0，即支持向量
    sVs=dataMat[svInd]
    labelSV=labelMat[svInd]
    print "there are %d Support Vectors" % shape(sVs)[0]
    m,n=shape(dataMat)
    errorCount=0
    for i in range(m):
        kernelEval=kernelTrans(sVs,dataMat[i,:],('rbf',k1))
        predict=kernelEval.T*multiply(labelSV,alphas[svInd])+b
        if sign(predict)!=sign(labelArr[i]):
            errorCount+=1
    print "the training error rate is: %f"%(float(errorCount)/m)
    dataArr,labelArr=loadDataSet('testSetRBF2.txt')
    errorCount=0
    dataMat=mat(dataArr);labelMat=mat(labelArr).transpose()
    m,n=shape(dataMat)
    for i in range(m):
        kernelEval=kernelTrans(sVs,dataMat[i,:],('rbf',k1))
        predict=kernelEval.T*multiply(labelSV,alphas[svInd])+b
        if sign(predict)!=sign(labelArr[i]):
            errorCount+=1
    print " the test error rate is: %f"%(float(errorCount)/m)

In [33]:
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

In [35]:
def loadImages(dirName):
    from os import listdir
    hwLabels=[]
    trainingFileList=listdir(dirName)
    m=len(trainingFileList)
    trainingMat=zeros((m,1024))
    for i in range(m):
        fileNameStr=trainingFileList[i]
        fileStr=fileNameStr.split('.')[0]
        classNumStr=int(fileStr.split('_')[0])
        if classNumStr==9:#9为-1，其他为1,svm只能二分类为-1或1，所以需转换
            hwLabels.append(-1)
        else:
            hwLabels.append(1)
        trainingMat[i,:]=img2vector('%s/%s'%(dirName,fileNameStr))
    return trainingMat,hwLabels

In [38]:
def testDigits(kTup=('rbf',10)):
    dataArr,labelArr=loadImages('trainingDigits')
    b,alphas=smoP(dataArr,labelArr,200,0.0001,10000,kTup)
    dataMat=mat(dataArr);labelMat=mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=dataMat[svInd]
    labelSV=labelMat[svInd]
    print "there are %d Support Vectors"%shape(sVs)[0]
    m,n=shape(dataMat)
    errorCount=0
    for i in range(m):
        kernelEval=kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T*multiply(labelSV,alphas[svInd])+b
        if sign(predict)!=sign(labelArr[i]):
            errorCount+=1
    print "the training error rate is: %f"%(float(errorCount)/m)
    dataArr,labelArr=loadImages('testDigits')
    errorCount=0
    datMat=mat(dataArr);labelMat=mat(labelArr).transpose()
    m,n=shape(datMat)
    for i in range(m):
        kernelEval=kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T*multiply(labelSV,alphas[svInd])+b
        if sign(predict)!=sign(labelArr[i]):
            errorCount+=1
    print "the test error rate is: %f"%(float(errorCount)/m)

In [ ]:
testDigits(('rbf',20))

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 2
fullSet, iter: 0 i:2, pairs changed 3
fullSet, iter: 0 i:3, pairs changed 4
fullSet, iter: 0 i:4, pairs changed 5
fullSet, iter: 0 i:5, pairs changed 6
fullSet, iter: 0 i:6, pairs changed 7
fullSet, iter: 0 i:7, pairs changed 8
L==H
fullSet, iter: 0 i:8, pairs changed 8
L==H
fullSet, iter: 0 i:9, pairs changed 8
L==H
fullSet, iter: 0 i:10, pairs changed 8
L==H
fullSet, iter: 0 i:11, pairs changed 8
L==H
fullSet, iter: 0 i:12, pairs changed 8
L==H
fullSet, iter: 0 i:13, pairs changed 8
L==H
fullSet, iter: 0 i:14, pairs changed 8
L==H
fullSet, iter: 0 i:15, pairs changed 8
L==H
fullSet, iter: 0 i:16, pairs changed 8
L==H
fullSet, iter: 0 i:17, pairs changed 8
L==H
fullSet, iter: 0 i:18, pairs changed 8
L==H
fullSet, iter: 0 i:19, pairs changed 8
L==H
fullSet, iter: 0 i:20, pairs changed 8
L==H
fullSet, iter: 0 i:21, pairs changed 8
L==H
fullSet, iter: 0 i:22, pairs changed 8
L==H
fullSet, iter: 0 i:23, pairs chan